In [6]:
import torch.nn as nn
import torch

In [1]:
def validate_unet(model, test_dataloader, criterion, device):
    model.eval()  # Establece el modelo en modo de evaluación
    
    correct = 0
    total = 0
    
    # Inicializa un diccionario para contar los píxeles correctos de cada clase
    correct_per_class = {i: 0 for i in range(5)}  # Asume que tienes 5 clases
    
    with torch.no_grad():
        for images, masks in test_dataloader:
            
            # Permuta las dimensiones de las imágenes a (N, C, H, W)
            images = images.permute(0, 3, 1, 2)

            images, masks = images.to(device, dtype=torch.float), masks.to(device,  dtype=torch.long)
            
            #  Obtiene la salida del modelo
            outputs = model(images)
            
            # COnvierte las predicciones de probabilidad en valores de la clase
            predictions = torch.argmax(outputs, dim=1)
            
            # Realiza una comparación pixel a pixel entre prediccion y mascara, y nos devulve un tensor de booleanos
            matches = predictions == masks.to(device)
            
            # Suma todos los True, es decir, las coincidencias entre preddición y máscara
            correct += torch.sum(matches).item() 
            
            # Nº total de pixeles de todas las mascaras del lote
            total += masks.numel()
            
            # Cuenta los píxeles correctos para cada clase
            for i in range(5):  # Asume que tienes 5 clases
                correct_per_class[i] += torch.sum((predictions == i) & matches).item()
            
    model.train() 
    
    # Divide los pixeles bien clasificados entre todos los que hay y asi obtenemos el % de acierto
    accuracy = (correct / total) * 100.0
    
    return accuracy, correct_per_class